### A atividade de hoje é sobre uma base de dados que já utilizamos em projetos anteriores com outras ferramentas e tecnologias.
A base encontra-se já disponível anteriormente sobre funcionários, e a atividade é a seguinte:

1 - Baixe a base em sua máquina no formato .csv;
2 - Abra um novo colab e crie uma nova seção PySpark;
3 - Suba a base da máquina para seu ambiente Pyspark no Colab;
4 - Elabore Análises com filtros e agrupamentos aprendidos em PySpark na aula de ontem;
5 - Cada filtro ou agrupamento que você realizar deve estar acompanhado (no bloco de código do respectivo colab) de um comentário sobre qual  pergunta essas dados filtrados ou agrupados tentam responder sobre essa base;
6 - Cada filtro ou agrupamento deverá ser salvo em um objeto PySpark e depois convertido em um objeto Pandas, ao longo
do dia você deverá realizar no mínimo 20 tabelas resultantes dessas análises;
7 - Após isso, as tabelas pandas deverão ser enviadas diretamente do colab para arquivos csv.
8 - O formato de entrega é o próprio link, ou arquivo ypynb do colab

Bom trabalho,

Abraços

In [ ]:
# Importação da biblioteca pandas
import pandas as pd

In [ ]:
# Instalação dos requisitos para o PySpark
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://archive.apache.org/dist/spark/spark-3.1.1/spark-3.1.1-bin-hadoop3.2.tgz
!tar xf spark-3.1.1-bin-hadoop3.2.tgz
!pip install -q findspark

In [ ]:
 !pip install pyspark


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.3/317.3 MB 5.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.2-py2.py3-none-any.whl size=317812365 sha256=f83f2db1165b92417c9c4bf3448c97a970b40945b82705cd1493a59ed486ceb5
  Stored in directory: /root/.cache/pip/wheels/34/34/bd/03944534c44b677cd5859f248090daa9fb27b3c8f8e5f49574
Successfully built pyspark


In [ ]:
from pyspark.sql.functions import avg, round, F   # media, arredondar valores


In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
       .appName("Análise de Funcionários") \
       .getOrCreate()

# Carregar o arquivo CSV
df = spark.read.csv('/content/funcionarios.csv', header=True, inferSchema=True)
df.show()

+-----+--------+--------------------+-----------------+--------------------+-----------------+------------+----------------+------------------------------------------+--------------+----------------------------------------------+---------+----------------+------------------------------------+-----------------+--------------------+-------------------------+------------+------------+--------------+---------------------------------+--------------+--------------+--------------------------------+--------------------+------------------------------------+--------------------------+--------------------------------------+-------------------------+---------------------------------------------+------------------------+------------------------------------+--------------------------------+----------------------------+-----------------------------------+
|Idade|Demissão|  Viagem de negócios|Diárias recebidas|        Departamento|Distância de Casa|Escolaridade|Área de Formação|Número de contratos de 

### 1. Contagem de funcionários por departamento.
### Quantos funcionários existem em cada departamento?

In [ ]:
# Agrupar funcionários por departamento e contar
contagem_por_departamento = df.groupBy("Departamento").count()
# Converter para Pandas
contagem_por_departamento_pd = contagem_por_departamento.toPandas()
# Salvar em um arquivo CSV
contagem_por_departamento_pd.to_csv('contagem_por_departamento.csv', index=False)
# Exibir os resultados
contagem_por_departamento.show()


+--------------------+-----+
|        Departamento|count|
+--------------------+-----+
|              Vendas|  446|
|Pesquisa e Desenv...|  961|
|   Recursos HumaNãos|   63|
+--------------------+-----+



### # 2. Média salarial por cargo
### Qual é a média salarial para cada cargo?

In [ ]:
# Agrupar por Cargo e calcular a media salarial
media_salario = df.groupBy("Cargo").agg(round(avg("Salário Mensal"), 2).alias("media_salario"))
media_salario_pd = media_salario.toPandas()
media_salario_pd.to_csv('media_salario_cargo.csv', index=False)
media_salario.show()

+--------------------+-------------+
|               Cargo|media_salario|
+--------------------+-------------+
|             Gerente|     15139.58|
|Diretor de manufa...|     14478.97|
| Diretor de pesquisa|     15080.55|
|Pesquisador cient...|     13927.11|
| Executivo de vendas|     14471.11|
|Técnico de Labora...|     14066.06|
|   Recursos HumaNãos|      13292.4|
|Reprentante de saúde|      14400.2|
|Representante de ...|      14278.2|
+--------------------+-------------+



###  3. Funcionários com salário acima da média
### Quais funcionários têm salário acima da média geral?

In [ ]:
# Calculando a média geral do salário
media_salario = df.select(avg("Salário Mensal")).first()[0]

# Filtrando os funcionários com salário acima da média arredondada
funcionarios_acima_media = df.filter(df["Salário Mensal"] > media_salario)

# Convertendo para Pandas
funcionarios_acima_media_pd = funcionarios_acima_media.toPandas()

# Salvando em um arquivo CSV
funcionarios_acima_media_pd.to_csv('funcionarios_acima_media.csv', index=False)

# Exibindo os resultados
df.select(avg("Salário Mensal")).show()
funcionarios_acima_media.show()


+-------------------+
|avg(Salário Mensal)|
+-------------------+
| 14313.103401360544|
+-------------------+

+-----+--------+--------------------+-----------------+--------------------+-----------------+------------+----------------+------------------------------------------+--------------+----------------------------------------------+---------+----------------+------------------------------------+-----------------+--------------------+-------------------------+------------+------------+--------------+---------------------------------+--------------+--------------+--------------------------------+--------------------+------------------------------------+--------------------------+--------------------------------------+-------------------------+---------------------------------------------+------------------------+------------------------------------+--------------------------------+----------------------------+-----------------------------------+
|Idade|Demissão|  Viagem de negócios

###  4. Contagem de funcionários por gênero
### Qual é a distribuição de gênero na empresa?

In [ ]:
# contagem de funcionarios por gênero
genero_contagem = df.groupBy("Gênero").count()
genero_contagem_pd = genero_contagem.toPandas()
genero_contagem_pd.to_csv("genero_contagem.csv", index=False)
genero_contagem.show()

+---------+-----+
|   Gênero|count|
+---------+-----+
| Feminino|  588|
|Masculino|  882|
+---------+-----+



###  5. Média de idade por departamento
### Qual é a média de idade dos funcionários em cada departamento?

In [ ]:
# Agrupar por departamento e calcular a media de idade
departamento_idade_media = df.groupBy("Departamento").agg(round(avg("Idade")).alias("departamento_idade_media"))
departamento_idade_media_pd = departamento_idade_media.toPandas()
departamento_idade_media_pd.to_csv('idade_media_por_departamento.csv', index=False)
departamento_idade_media.show()

+--------------------+------------------------+
|        Departamento|departamento_idade_media|
+--------------------+------------------------+
|              Vendas|                    37.0|
|Pesquisa e Desenv...|                    37.0|
|   Recursos HumaNãos|                    38.0|
+--------------------+------------------------+



### # 6. Quais funcionários têm mais de 5 anos de experiência na empresa?

In [ ]:
# 6. Quais funcionários têm mais de 5 anos de experiência na empresa?
anos_trabalho = df.filter(df["Total de anos trabalhados na empresa"] > 5)
anos_trabalho_pd = anos_trabalho.toPandas()
anos_trabalho_pd.to_csv('anos_trabalho.csv', index=False)
anos_trabalho.show()

+-----+--------+--------------------+-----------------+--------------------+-----------------+------------+----------------+------------------------------------------+--------------+----------------------------------------------+---------+----------------+------------------------------------+-----------------+--------------------+-------------------------+------------+------------+--------------+---------------------------------+--------------+--------------+--------------------------------+--------------------+------------------------------------+--------------------------+--------------------------------------+-------------------------+---------------------------------------------+------------------------+------------------------------------+--------------------------------+----------------------------+-----------------------------------+
|Idade|Demissão|  Viagem de negócios|Diárias recebidas|        Departamento|Distância de Casa|Escolaridade|Área de Formação|Número de contratos de 

###  7. Qual é a distribuição de funcionários por nível educacional?

In [ ]:
# Agrupar por nivel de escolaridade e contar o número de funcionários em cada nível
nivel_escolaridade = df.groupBy("Escolaridade").count()
nivel_escolaridade_pd = nivel_escolaridade.toPandas()
nivel_escolaridade_pd.to_csv('nivel_escolaridade.csv', index=False)
nivel_escolaridade.show()

+------------+-----+
|Escolaridade|count|
+------------+-----+
|           1|  170|
|           3|  572|
|           5|   48|
|           4|  398|
|           2|  282|
+------------+-----+



### 8. Pergunta: Quais funcionários não viajam?

In [ ]:
# Filtrar por coluna Viagem de negócios, selecionando não viaja.
empregados_nao_viajam = df.filter(df["Viagem de negócios"] == "Não Viaja")
empregados_nao_viajam_pd = empregados_nao_viajam.toPandas()
empregados_nao_viajam_pd.to_csv("empregados_nao_viajam", index=False)
empregados_nao_viajam.show()

+-----+--------+------------------+-----------------+--------------------+-----------------+------------+-----------------+------------------------------------------+--------------+----------------------------------------------+---------+----------------+------------------------------------+-----------------+--------------------+-------------------------+------------+------------+--------------+---------------------------------+--------------+--------------+--------------------------------+--------------------+------------------------------------+--------------------------+--------------------------------------+-------------------------+---------------------------------------------+------------------------+------------------------------------+--------------------------------+----------------------------+-----------------------------------+
|Idade|Demissão|Viagem de negócios|Diárias recebidas|        Departamento|Distância de Casa|Escolaridade| Área de Formação|Número de contratos de tr

### 9. Qual é a distribuição de funcionários por cargo e gênero?

In [ ]:
# Agrupar por cargo e gênero e contar o número de funcionários
distribuicao_cargo_genero = df.groupBy("Cargo", "Gênero").count()
# Converter para Pandas
distribuicao_cargo_genero_pd = distribuicao_cargo_genero.toPandas()
# Salvar em um arquivo CSV
distribuicao_cargo_genero_pd.to_csv('distribuicao_cargo_genero.csv', index=False)
distribuicao_cargo_genero.show()

+--------------------+---------+-----+
|               Cargo|   Gênero|count|
+--------------------+---------+-----+
|             Gerente| Feminino|   47|
|Diretor de manufa...|Masculino|   73|
|   Recursos HumaNãos| Feminino|   16|
|Representante de ...| Feminino|   38|
| Diretor de pesquisa| Feminino|   33|
|Representante de ...|Masculino|   45|
|Pesquisador cient...| Feminino|  114|
|Reprentante de saúde|Masculino|   80|
|   Recursos HumaNãos|Masculino|   36|
|Técnico de Labora...|Masculino|  174|
|             Gerente|Masculino|   55|
|Técnico de Labora...| Feminino|   85|
|Reprentante de saúde| Feminino|   51|
|Pesquisador cient...|Masculino|  178|
| Executivo de vendas|Masculino|  194|
| Diretor de pesquisa|Masculino|   47|
|Diretor de manufa...| Feminino|   72|
| Executivo de vendas| Feminino|  132|
+--------------------+---------+-----+



### 10. Qual é a média de horas trabalhadas por departamento?

In [ ]:
# Agrupar por departamento e calcular a média do score de performance
media_performance_por_departamento = df.groupBy("Departamento").agg(avg("Score de Performance").alias("media_performance"))

# Converter para Pandas
media_performance_por_departamento_pd = media_performance_por_departamento.toPandas()
# Salvar em um arquivo CSV
media_performance_por_departamento_pd.to_csv('media_performance_por_departamento.csv', index=False)
media_performance_por_departamento.show()



+--------------------+------------------+
|        Departamento| media_performance|
+--------------------+------------------+
|              Vendas|3.1367713004484306|
|Pesquisa e Desenv...| 3.162330905306972|
|   Recursos HumaNãos| 3.142857142857143|
+--------------------+------------------+



### 11. Qual é a distribuição de funcionários por estado civil?

In [ ]:
# Agrupar funcionários por estado civil e contar
contagem_por_estado_civil = df.groupBy("Estado Civil").count()
# Converter para Pandas
contagem_por_estado_civil_pd = contagem_por_estado_civil.toPandas()
# Salvar em um arquivo CSV
contagem_por_estado_civil_pd.to_csv('contagem_por_estado_civil.csv', index=False)
contagem_por_estado_civil.show()


+------------+-----+
|Estado Civil|count|
+------------+-----+
|  Divorciado|  327|
|    Solteiro|  470|
|      Casado|  673|
+------------+-----+



### 12. Qual é a distribuição de funcionários por faixa etária?

In [ ]:
from pyspark.sql import functions as F
# Definindo a faixa etária
faixa_etaria_bins = [20, 30, 40, 50, 60, 70]
faixa_etaria_labels = ['18-29', '30-39', '40-49', '50-59', '60-69']
# Criando a coluna de faixa etária
df = df.withColumn("faixa_etaria", F.when(df.Idade < 30, '18-29')
                   .when((df.Idade >= 30) & (df.Idade < 40), '30-39')
                   .when((df.Idade >= 40) & (df.Idade < 50), '40-49')
                   .when((df.Idade >= 50) & (df.Idade < 60), '50-59')
                   .otherwise('60-69'))
# Agrupando e contando funcionários por faixa etária
contagem_por_faixa_etaria = df.groupBy("faixa_etaria").count()
# Convertendo para Pandas
contagem_por_faixa_etaria_pd = contagem_por_faixa_etaria.toPandas()
# Salvando em um arquivo CSV
contagem_por_faixa_etaria_pd.to_csv('contagem_por_faixa_etaria.csv', index=False)
contagem_por_faixa_etaria.show()



+------------+-----+
|faixa_etaria|count|
+------------+-----+
|       30-39|  622|
|       60-69|    5|
|       40-49|  349|
|       18-29|  326|
|       50-59|  168|
+------------+-----+



### 13. Qual é a média salarial por gênero?

In [ ]:
# Calculando a média salarial por gênero
media_salarial_por_genero = df.groupBy("Gênero").agg(F.round(F.avg("Salário Mensal"), 2).alias("media_salarial"))
# Convertendo para Pandas
media_salarial_por_genero_pd = media_salarial_por_genero.toPandas()
# Salvando em um arquivo CSV
media_salarial_por_genero_pd.to_csv('media_salarial_por_genero.csv', index=False)
media_salarial_por_genero.show()


+---------+--------------+
|   Gênero|media_salarial|
+---------+--------------+
| Feminino|       14674.6|
|Masculino|      14072.11|
+---------+--------------+



### 14. Contagem de funcionários por tempo na empresa

In [ ]:
# Agrupar por tempo de serviço e contar o número de funcionários
contagem_por_tempo_empresa = df.groupBy("Total de anos trabalhados na empresa").count()
# Convertendo para Pandas
contagem_por_tempo_empresa_pd = contagem_por_tempo_empresa.toPandas()
# Salvando em um arquivo CSV
contagem_por_tempo_empresa_pd.to_csv('contagem_por_tempo_empresa.csv', index=False)
contagem_por_tempo_empresa.show()

+------------------------------------+-----+
|Total de anos trabalhados na empresa|count|
+------------------------------------+-----+
|                                  31|    3|
|                                  34|    1|
|                                  27|    2|
|                                  26|    4|
|                                  12|   14|
|                                  22|   15|
|                                   1|  171|
|                                  13|   24|
|                                   6|   76|
|                                  16|   12|
|                                   3|  128|
|                                  20|   27|
|                                  40|    1|
|                                   5|  196|
|                                  19|   11|
|                                  15|   20|
|                                  37|    1|
|                                   9|   82|
|                                  17|    9|
|         

### 15. Funcionários com cargo de liderança e ativos?


In [ ]:
# Filtrando funcionários que ocupam cargos de liderança e que não pediram demissão
cargos_lideranca_ativos = df.filter(
    (df["Cargo"].isin(["Gerente", "Diretor de pesquisa", "Diretor de manufatura", "Executivo de vendas"])) &
    (df["Demissão"] == "Não")
)
# Convertendo para Pandas
cargos_lideranca_ativos_pd = cargos_lideranca_ativos.toPandas()
# Salvando em um arquivo CSV
cargos_lideranca_ativos_pd.to_csv('cargos_lideranca_ativos.csv', index=False)
cargos_lideranca_ativos.show()



+-----+--------+--------------------+-----------------+--------------------+-----------------+------------+----------------+------------------------------------------+--------------+----------------------------------------------+---------+----------------+------------------------------------+-----------------+--------------------+-------------------------+------------+------------+--------------+---------------------------------+--------------+--------------+--------------------------------+--------------------+------------------------------------+--------------------------+--------------------------------------+-------------------------+---------------------------------------------+------------------------+------------------------------------+--------------------------------+----------------------------+-----------------------------------+------------+
|Idade|Demissão|  Viagem de negócios|Diárias recebidas|        Departamento|Distância de Casa|Escolaridade|Área de Formação|Número de 

 ### 16. Funcionários que foram Demitidos:



In [ ]:
# Filtrando funcionários que foram demitidos
funcionarios_demitidos = df.filter(df["Demissão"] == "Sim")
# Convertendo para Pandas
funcionarios_demitidos_pd = funcionarios_demitidos.toPandas()
# Salvando em um arquivo CSV
funcionarios_demitidos_pd.to_csv('funcionarios_demitidos.csv', index=False)
funcionarios_demitidos.show()


+-----+--------+--------------------+-----------------+--------------------+-----------------+------------+-----------------+------------------------------------------+--------------+----------------------------------------------+---------+----------------+------------------------------------+-----------------+--------------------+-------------------------+------------+------------+--------------+---------------------------------+--------------+--------------+--------------------------------+--------------------+------------------------------------+--------------------------+--------------------------------------+-------------------------+---------------------------------------------+------------------------+------------------------------------+--------------------------------+----------------------------+-----------------------------------+------------+
|Idade|Demissão|  Viagem de negócios|Diárias recebidas|        Departamento|Distância de Casa|Escolaridade| Área de Formação|Número d

### 17. Funcionários que participaram de treinamentos no último ano:

In [ ]:
# Filtrando funcionários que participaram de algum treinamento
participantes_treinamento = df.filter(df["Qtde de treinamentos realizados no último ano"] > 0)
# Convertendo para Pandas
participantes_treinamento_pd = participantes_treinamento.toPandas()
# Salvando em um arquivo CSV
participantes_treinamento_pd.to_csv('participantes_treinamento.csv', index=False)
participantes_treinamento.show()


+-----+--------+--------------------+-----------------+--------------------+-----------------+------------+----------------+------------------------------------------+--------------+----------------------------------------------+---------+----------------+------------------------------------+-----------------+--------------------+-------------------------+------------+------------+--------------+---------------------------------+--------------+--------------+--------------------------------+--------------------+------------------------------------+--------------------------+--------------------------------------+-------------------------+---------------------------------------------+------------------------+------------------------------------+--------------------------------+----------------------------+-----------------------------------+------------+
|Idade|Demissão|  Viagem de negócios|Diárias recebidas|        Departamento|Distância de Casa|Escolaridade|Área de Formação|Número de 

### 18. Funcionários que não participaram de treinamentos no último ano:

In [ ]:
# Filtrando funcionários que não participaram de nenhum treinamento
funcionarios_sem_treinamento = df.filter(df["Qtde de treinamentos realizados no último ano"] == 0)
# Convertendo para Pandas
funcionarios_sem_treinamento_pd = funcionarios_sem_treinamento.toPandas()
# Salvando em um arquivo CSV
funcionarios_sem_treinamento_pd.to_csv('funcionarios_sem_treinamento.csv', index=False)
funcionarios_sem_treinamento.show()


+-----+--------+--------------------+-----------------+--------------------+-----------------+------------+----------------+------------------------------------------+--------------+----------------------------------------------+---------+----------------+------------------------------------+-----------------+--------------------+-------------------------+------------+------------+--------------+---------------------------------+--------------+--------------+--------------------------------+--------------------+------------------------------------+--------------------------+--------------------------------------+-------------------------+---------------------------------------------+------------------------+------------------------------------+--------------------------------+----------------------------+-----------------------------------+------------+
|Idade|Demissão|  Viagem de negócios|Diárias recebidas|        Departamento|Distância de Casa|Escolaridade|Área de Formação|Número de 

### 19. Contagem de funcionários por satisfação com o trabalho:

In [ ]:
# Qual é a distribuição de funcionários por Satisfação com o trabalho?
satisfacao_trabalho = df.groupBy("Satisfação com o trabalho").count()
satisfacao_trabalho_pd = satisfacao_trabalho.toPandas()
satisfacao_trabalho_pd.to_csv('satisfacao_trabalho.csv', index=False)
satisfacao_trabalho.show()


+-------------------------+-----+
|Satisfação com o trabalho|count|
+-------------------------+-----+
|                        1|  289|
|                        3|  442|
|                        4|  459|
|                        2|  280|
+-------------------------+-----+



### 20. Funcionários com mais de 20 anos na empresa:



In [ ]:
# 20. Pergunta: Quantos funcionários têm mais de 15 anos na empresa?
# Filtrando funcionários com mais de 15 anos na empresa
funcionarios_longa_duracao = df.filter(df["Total de anos trabalhados na empresa"] > 20)
# Convertendo para Pandas
funcionarios_longa_duracao_pd = funcionarios_longa_duracao.toPandas()
# Salvando em um arquivo CSV
funcionarios_longa_duracao_pd.to_csv('funcionarios_longa_duracao.csv', index=False)
funcionarios_longa_duracao.show()


+-----+--------+--------------------+-----------------+--------------------+-----------------+------------+----------------+------------------------------------------+--------------+----------------------------------------------+---------+----------------+------------------------------------+-----------------+--------------------+-------------------------+------------+------------+--------------+---------------------------------+--------------+--------------+--------------------------------+--------------------+------------------------------------+--------------------------+--------------------------------------+-------------------------+---------------------------------------------+------------------------+------------------------------------+--------------------------------+----------------------------+-----------------------------------+------------+
|Idade|Demissão|  Viagem de negócios|Diárias recebidas|        Departamento|Distância de Casa|Escolaridade|Área de Formação|Número de 